In [1]:
%reload_ext autoreload
%autoreload 2

# Load the data

In [2]:
from result_saver import SaverProvider
provider = SaverProvider()

In [9]:
from Scratch import metadata_loader
import numpy as np
from tqdm import tqdm

DEVICE = "ibm_sherbrooke"
LOGICAL = str(0)
pairs_to_process = [(10, 10), (20, 20), (30, 35), (40, 35), (55, 35)]

# Load the metadata
md = metadata_loader(True, True)
md = md[md["job_status"] == "JobStatus.DONE"]
md = md[md["code"] == "RepetitionCodeCircuit"]
md = md.dropna(subset=["rounds"])
md = md[md["meas_level"] == 1]
md['rounds'] = md['rounds'].astype(int)
md['distance'] = md['distance'].astype(int)

md = md[md["backend_name"] == DEVICE]
md = md[md["logical"] == LOGICAL]

md[:2]

,creation_date,notebook_name,backend_name,job_id,tags,meas_level,shots,num_qubits,job_status,execution_date,code,distance,rounds,logical,descr,resets,xbasis,path_info
5315,2024-03-23 17:56:21.062000+01:00,Sherbrooke_RepCodes_jobs,ibm_sherbrooke,cqzgkd5ktf3g00883v1g,"[Subset 52, 100 rounds, 0 log, xbasis=True]",1.0,768.0,NaN,JobStatus.DONE,2024-03-23 18:34:16.754492+01:00,RepetitionCodeCircuit,52,100,0,subset RepCodes,NaN,True,"{'mean_gate_error': 0.008345314216382, 'min_ga..."
5314,2024-03-23 17:56:05.176000+01:00,Sherbrooke_RepCodes_jobs,ibm_sherbrooke,cqzgk95dvs8g008j6ep0,"[Subset 52, 100 rounds, 0 log, xbasis=True]",1.0,768.0,NaN,JobStatus.DONE,2024-03-23 18:33:46.104916+01:00,RepetitionCodeCircuit,52,100,0,subset RepCodes,NaN,True,"{'mean_gate_error': 0.008345314216382, 'min_ga..."


# Group job ids by closest calibration date

In [4]:
from datetime import datetime
from Scratch import find_closest_calib_jobs
import pandas as pd

jobs_by_calibration_date = {}
for index, row in md.iterrows():
    if (row['distance'], row['rounds']) not in pairs_to_process:
        continue
    job_id = row['job_id']

    _, _, calib_creation_date = find_closest_calib_jobs(tobecalib_job=job_id, verbose=False)

    if calib_creation_date not in jobs_by_calibration_date.keys():
        jobs_by_calibration_date[calib_creation_date] = [job_id]
    else:
        jobs_by_calibration_date[calib_creation_date].append(job_id)

# Takes 21s

In [5]:
jobs_by_calibration_date

{datetime.datetime(2023, 11, 9, 16, 47, 14, 556645, tzinfo=datetime.timezone.utc): ['cnn25s724wx0008f9kz0',
  'cn6hm589recg008x0jvg',
  'cn6hk3mxhnxg008djq30',
  'cn6h9gpss5h00087k140',
  'cn6h85r62r90008814pg',
  'cn6h75464yf0008g84h0',
  'cn6h5tfss5h00087k0w0',
  'cn6h4f164yf0008g84ag',
  'cn6h3dnxhnxg008djny0',
  'cn6h20ranbvg008dab10',
  'cn6h0mjss5h00087k0a0',
  'cn6gzk664yf0008g83f0',
  'cn6gy68ss5h00087jzsg',
  'cn6gwrkss5h00087jzpg',
  'cn6gvnyxhnxg008djn4g',
  'cn6grptss5h00087jz80',
  'cn6gqa59recg008x0ghg',
  'cn6gnxzss5h00087jyz0',
  'cn6gmvb62r90008812sg',
  'cn6gee1anbvg008da9k0',
  'cn6gczv9recg008x0fng',
  'cn6gbx79recg008x0ff0',
  'cn6gaga62r90008811tg',
  'cn6g93cxhnxg008djkwg',
  'cn6g81rss5h00087jxeg',
  'cn6g6mtxhnxg008djkkg',
  'cn6g57c62r900088110g',
  'cn6g44862r90008810ng',
  'cn6bdpevayrg008ermxg',
  'cn6bca1rmwhg008k4xx0',
  'cn6bb7wrmwhg008k4xa0',
  'cn6b9pprmwhg008k4x60',
  'cn6b89s3r3vg008fcvzg',
  'cn6b76wrmwhg008k4x20',
  'cn6b5v7p1am0008qeza0',
  'cn6b4

# Decode the data and save results

In [8]:
import json

import pymatching
import stim 
from soft_info import get_noise_dict_from_backend, get_avgs_from_dict, find_longest_path_in_hex, get_repcode_IQ_map
from soft_info import RepetitionCodeStimCircuit
import cpp_soft_info
from Scratch import load_calibration_memory, create_or_load_kde_grid

_DETAILED = False

rel_error = 1
_RESETS = False
nb_intervals = -1

# KDE BANDWIDTHS
lin = [0.4, 0.7, 20]
num_points = 51

# lin = [0.6, 0.7, 1]
# num_points = 2

bandwidths = np.linspace(lin[0], lin[1], lin[2])




for calib_date in jobs_by_calibration_date.keys():

    # get the noise dict of that date
    noise_dict = get_noise_dict_from_backend(provider, DEVICE, date = calib_date)

    # get the KDE of that date
    all_memories = load_calibration_memory(provider, tobecalib_backend=DEVICE, other_date=calib_date)
    kde_dict = cpp_soft_info.get_KDEs(all_memories, bandwidths, relError=rel_error, absError=-1, num_points=51) # Less num_points bcs just 1 bandwidth


    # get the longest path of that device
    longest_path, _, _ = find_longest_path_in_hex(provider.get_backend(DEVICE))

    # decode each job of that date
    for job_id in tqdm(jobs_by_calibration_date[calib_date], desc=f"Decoding jobs of {calib_date} calibration"):
        distance = md[md["job_id"] == job_id]["distance"].values[0]
        rounds = md[md["job_id"] == job_id]["rounds"].values[0]
        IQ_data = provider.retrieve_job(job_id).result().get_memory()

        # Get the layout for the avgs
        bounded_path = longest_path[:2 * distance - 1]
        layout = bounded_path[1::2] + bounded_path[::2]
        qubit_mapping = get_repcode_IQ_map(layout, rounds)

        # Get the avgs
        avgs = get_avgs_from_dict(noise_dict, layout)
        # noise_list = [avgs["two_gate"], avgs["single_gate"], avgs["t1_err"], avgs["t2_err"]]
        noise_list = [avgs["two_gate"], avgs["single_gate"], 3e-2, 3e-2]
        readout = avgs["readout"]
        soft_multiplicator = 0.5
        noise_list += [readout, readout*(1-soft_multiplicator), readout*soft_multiplicator]    
        #[twog_err, sglg_err, t1_err, t2_err, readout_err, hard_err, soft_err]

        # Stim model
        subsampling = False
        code = RepetitionCodeStimCircuit(distance, rounds, xbasis=False, resets=_RESETS, 
                                        noise_list=noise_list, subsampling=subsampling)
        model = code.circuits[LOGICAL].detector_error_model(decompose_errors=False)

        result_soft, result_hard = cpp_soft_info.decode_all_kde(model, IQ_data, rounds, int(LOGICAL), _RESETS,
                                                                qubit_mapping, kde_dict, relError=rel_error, 
                                                                absError=-1, nb_intervals=nb_intervals)


        result_kde_json = {
            "decoding": "kde",
            "num_errors": result_soft.num_errors,
            "additional_info": {            
                "rel_error": rel_error,
                "bandwidth_linspace": lin,
                "num_points_bandwidths": num_points
            },
        }


        result_informed_json = {
            "decoding": "informed",
            "num_errors": result_hard.num_errors,
        }



        ############# SAVING THE RESULT ##############

        with open(f"../results/softInfo_vs_informed_v2.json", "r") as f:
            data = json.load(f)
            if job_id not in data.keys():
                data[job_id] = [result_kde_json, result_informed_json]
            else:
                data[job_id].append(result_kde_json)
                data[job_id].append(result_informed_json)
        
        with open(f"../results/softInfo_vs_informed_v2.json", "w") as f:
            json.dump(data, f, indent=4)



16:11:26 Warning: Z0 decoding. Negative T2 error -0.03 % for qubit 42, setting to 0.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 52
Found jobs for backend ibm_sherbrooke with closest execution date 2023-11-09 20:02:27+00:00.


Decoding jobs of 2023-11-09 16:47:14.556645+00:00 calibration: 100%|██████████| 46/46 [08:36<00:00, 11.22s/it]


Found jobs for backend ibm_sherbrooke with closest execution date 2023-10-27 08:32:22.841567+00:00.


Decoding jobs of 2023-10-27 07:46:44.189709+00:00 calibration: 100%|██████████| 7/7 [01:16<00:00, 10.95s/it]


import json

import pymatching
import stim 
from soft_info import get_noise_dict_from_backend, get_avgs_from_dict, find_longest_path_in_hex, get_repcode_IQ_map
import cpp_soft_info
from Scratch import load_calibration_memory, create_or_load_kde_grid

_DETAILED = False

rel_error = -1
_RESETS = False
nb_intervals = -1

# KDE BANDWIDTHS
lin = [0.4, 0.7, 20]
num_points = 51

# lin = [0.6, 0.7, 1]
# num_points = 2

bandwidths = np.linspace(lin[0], lin[1], lin[2])
print("bandwidths:", bandwidths)




for calib_date in jobs_by_calibration_date.keys():

    # get the noise dict of that date
    noise_dict = get_noise_dict_from_backend(provider, DEVICE, date = calib_date)

    # get the KDE of that date
    all_memories = load_calibration_memory(provider, tobecalib_backend=DEVICE, other_date=calib_date)
    kde_dict = cpp_soft_info.get_KDEs(all_memories, bandwidths, relError=rel_error, absError=-1, num_points=51) # Less num_points bcs just 1 bandwidth

    # get the longest path of that device
    longest_path, _, _ = find_longest_path_in_hex(provider.get_backend(DEVICE))

    # decode each job of that date
    for job_id in tqdm(jobs_by_calibration_date[calib_date], desc=f"Decoding jobs of {calib_date} calibration"):
        distance = md[md["job_id"] == job_id]["distance"].values[0]
        rounds = md[md["job_id"] == job_id]["rounds"].values[0]
        IQ_data = provider.retrieve_job(job_id).result().get_memory()

        # Get the layout for the avgs
        bounded_path = longest_path[:2 * distance - 1]
        layout = bounded_path[1::2] + bounded_path[::2]
        qubit_mapping = get_repcode_IQ_map(layout, rounds)

        # Get the avgs
        avgs = get_avgs_from_dict(noise_dict, layout)

        # Initialize the stim model
        circuit = stim.Circuit.generated("repetition_code:memory",
                                distance=distance,
                                rounds=rounds,
                                after_clifford_depolarization=avgs["two_gate"], #two-qubit-fidelity,
                                after_reset_flip_probability=0 if not _RESETS else avgs["readout"], #reset error
                                before_measure_flip_probability=avgs["readout"], #measurement error,
                                before_round_data_depolarization=avgs["idle"]) #idle error)
        model = circuit.detector_error_model(decompose_errors=False)
        matching = pymatching.Matching.from_detector_error_model(model)

        ############# DECODING ##############


        result_kde = cpp_soft_info.decode_IQ_kde(model, IQ_data, rounds, int(LOGICAL), _RESETS, 
                                         qubit_mapping, kde_dict, _DETAILED, relError=rel_error, absError=-1,
                                         nb_intervals=nb_intervals)
        
        result_kde_json = {
            "decoding": "kde_relError",
            "num_errors": result_kde.num_errors,
            "additional_info": {            
                "rel_error": rel_error,
                "bandwidth_linspace": lin,
                "num_points_bandwidths": num_points
            },
        }


        ############# SAVING THE RESULT ##############

        with open(f"../results/softInfo_vs_informed.json", "r") as f:
            data = json.load(f)
            if job_id not in data.keys():
                data[job_id] = [result_kde_json]
            else:
                data[job_id].append(result_kde_json)
        
        with open(f"../results/softInfo_vs_informed.json", "w") as f:
            json.dump(data, f, indent=4)



import json

import pymatching
import stim 
from soft_info import get_noise_dict_from_backend, get_avgs_from_dict, find_longest_path_in_hex, get_repcode_IQ_map
import cpp_soft_info
from Scratch import load_calibration_memory, create_or_load_kde_grid

_DETAILED = False

rel_error = 1
_RESETS = False
nb_intervals = -1

# KDE BANDWIDTHS
# lin = [0.4, 0.7, 20]
# num_points = 51

lin = [0.6, 0.7, 1]
num_points = 2

bandwidths = np.linspace(lin[0], lin[1], lin[2])
print("bandwidths:", bandwidths)




for calib_date in jobs_by_calibration_date.keys():

    # get the noise dict of that date
    noise_dict = get_noise_dict_from_backend(provider, DEVICE, date = calib_date)

    # get the KDE of that date
    all_memories = load_calibration_memory(provider, tobecalib_backend=DEVICE, other_date=calib_date)
    kde_dict = cpp_soft_info.get_KDEs(all_memories, bandwidths, relError=rel_error, absError=-1, num_points=51) # Less num_points bcs just 1 bandwidth

    # get the longest path of that device
    longest_path, _, _ = find_longest_path_in_hex(provider.get_backend(DEVICE))

    # decode each job of that date
    for job_id in tqdm(jobs_by_calibration_date[calib_date], desc=f"Decoding jobs of {calib_date} calibration"):
        distance = md[md["job_id"] == job_id]["distance"].values[0]
        rounds = md[md["job_id"] == job_id]["rounds"].values[0]
        IQ_data = provider.retrieve_job(job_id).result().get_memory()

        # Get the layout for the avgs
        bounded_path = longest_path[:2 * distance - 1]
        layout = bounded_path[1::2] + bounded_path[::2]
        qubit_mapping = get_repcode_IQ_map(layout, rounds)

        # Get the avgs
        avgs = get_avgs_from_dict(noise_dict, layout)

        # Initialize the stim model
        circuit = stim.Circuit.generated("repetition_code:memory",
                                distance=distance,
                                rounds=rounds,
                                after_clifford_depolarization=avgs["two_gate"], #two-qubit-fidelity,
                                after_reset_flip_probability=0 if not _RESETS else avgs["readout"], #reset error
                                before_measure_flip_probability=avgs["readout"], #measurement error,
                                before_round_data_depolarization=avgs["idle"]) #idle error)
        model = circuit.detector_error_model(decompose_errors=False)
        matching = pymatching.Matching.from_detector_error_model(model)

        ############# DECODING ##############


        result_kde = cpp_soft_info.decode_IQ_kde(model, IQ_data, rounds, int(LOGICAL), _RESETS, 
                                         qubit_mapping, kde_dict, _DETAILED, relError=rel_error, absError=-1,
                                         nb_intervals=nb_intervals)
        
        result_kde_json = {
            "decoding": "kde_bandwidth",
            "num_errors": result_kde.num_errors,
            "additional_info": {            
                "rel_error": rel_error,
                "bandwidth_linspace": lin,
                "num_points_bandwidths": num_points
            },
        }


        ############# SAVING THE RESULT ##############

        with open(f"../results/softInfo_vs_informed.json", "r") as f:
            data = json.load(f)
            if job_id not in data.keys():
                data[job_id] = [result_kde_json]
            else:
                data[job_id].append(result_kde_json)
        
        with open(f"../results/softInfo_vs_informed.json", "w") as f:
            json.dump(data, f, indent=4)

